In [40]:
from time import time
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from gensim.models import word2vec

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [41]:
df = pd.read_csv("./labeled_rutoxic.csv", delimiter=',', header=0, names=['sentence', 'label'])

print('В наборе предложений: \n',df.shape[0])

В наборе предложений: 
 14412


In [42]:
print('toxic:', df[df['label'] > 0]['label'].count())
print('not toxic:', df[df['label'] < 1]['label'].count())

toxic: 4826
not toxic: 9586


## Разбиение на тестовые и обучающие

In [43]:
X = df.iloc[:,0]#
y = df.iloc[:,1]#

train , test , y_train, y_test = train_test_split(X, y, test_size=0.3) # и отдаем 30% на тест, остальное на обучен

## Разбиение на слова с помощью модели Word2Vec

In [44]:
import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Метод преобразования текста в слова
def text_to_words(raw_text, remove_stopwords=False):
    # Удаление лишних символов, оставляем только буквы и цифры
    letters_and_numbers_only = re.sub("[^0-9а-яА-Я]", " ", raw_text)

    # Приведение к нижнему регистру и токенизация
    words = letters_and_numbers_only.lower().split()

    # Удаление стоп-слов, если требуется
    if remove_stopwords:
        stops = set(stopwords.words("russian"))
        words = [w for w in words if not w in stops]

    # Лемматизация
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

sentences_train = train.apply(text_to_words, remove_stopwords=False)
sentences_test = test.apply(text_to_words, remove_stopwords=False)
print("\nSentences_train:")
print(sentences_train[:10])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



Sentences_train:
3091     [я, в, курсе, за, каргокульты, их, самолеты, п...
13633    [я, тебе, ещ, раз, говорю, ты, баран, тупороты...
3050                         [шаболда, уровня, легалпорно]
851      [что, делать, свети, ться, ться, ться, ться, т...
9249     [вы, определитесь, нужно, производство, нужны,...
9161     [вы, не, о, том, говорите, контроль, подразуме...
943             [н, е, к, у, к, о, л, д, ы, и, р, а, з, у]
2715     [жидам, нельзя, доверять, жид, всегда, будет, ...
928      [эта, та, история, что, наука, или, та, что, к...
3823      [узкий, хват, в, таком, случае, максимум, травм]
Name: sentence, dtype: object


In [59]:

num_features = 300
min_word_count = 40
num_workers = 4
context = 20
downsampling = 1e-3
model = word2vec.Word2Vec(sentences_train, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)
print(sentences_train[:1])
print(model.wv[1].shape)
print(model.wv[1])

3091    [я, в, курсе, за, каргокульты, их, самолеты, п...
Name: sentence, dtype: object
(300,)
[ 6.04244834e-03 -3.04302629e-02  1.55952051e-01  4.69616912e-02
  3.81012186e-02 -1.87213257e-01 -2.26431623e-01  1.30195901e-01
  3.28672789e-02 -3.05439085e-02  2.54044503e-01 -1.85467988e-01
  4.80611175e-02 -1.88185960e-01 -1.51656017e-01 -1.63152829e-01
  6.74176887e-02 -9.97117255e-03  1.30403280e-01  3.18484336e-01
 -2.52727699e-02 -1.14951804e-01  1.36236295e-01 -1.77274607e-02
  2.60671735e-01 -7.28831142e-02 -1.42565846e-01 -2.27121815e-01
 -6.01600222e-02 -4.56124067e-01  2.20427662e-01 -1.52850151e-01
  1.73288122e-01 -4.15189490e-02  1.60807241e-02 -1.38811588e-01
 -1.46530792e-01 -2.41404161e-01  1.31483218e-02 -6.70249388e-02
  1.59195423e-01  2.38658994e-01 -1.26081795e-01 -2.90445417e-01
  1.27652809e-01  1.56284243e-01  4.05189656e-02 -9.78976488e-02
  3.47906798e-02 -1.64468065e-01  1.87533617e-01  1.65780172e-01
 -2.70791054e-01 -6.12659231e-02 -1.53882220e-01  2.60366321

In [46]:
# получение векторного представления
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    index2word_set = set(model.wv.index_to_key)

    for word in words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model.wv[word])

    if nwords == 0:
        nwords = 1
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# получение среднего векторного простнраства для предложения
def getAvgFeatureVecs(reviews, model, num_features):
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    counter = 0
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
    return reviewFeatureVecs

f_matrix_train = getAvgFeatureVecs(sentences_train, model, num_features)
f_matrix_test = getAvgFeatureVecs(sentences_test, model, num_features)
print("Числовые вектора предложений полученные из усреднения его векторов слов:")
print(f_matrix_train[:10])

Числовые вектора предложений полученные из усреднения его векторов слов:
[[-0.03328236 -0.00121379  0.17288294 ... -0.16263957  0.1967497
  -0.18197   ]
 [-0.09652852  0.19179352  0.01154332 ... -0.07593644  0.12609579
   0.01830778]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [-0.0768503   0.19889422  0.01900334 ... -0.11224172  0.121709
  -0.00285473]
 [-0.0761342   0.17978963  0.03542842 ... -0.1162403   0.12634583
  -0.01343997]
 [-0.02748077  0.08471926  0.1123777  ... -0.15854587  0.17173629
  -0.12328024]]


## Обучение  MLPClassifier

In [47]:
model = []
#adam солвер это стохастически градиентный оптимизатор
m = MLPClassifier(solver='adam', hidden_layer_sizes=(100,50,20), random_state=1)
model.append(m)

print(model)

[MLPClassifier(hidden_layer_sizes=(100, 50, 20), random_state=1)]


In [48]:
batch_size = 200
total_rows = f_matrix_train.shape[0]
duration = 0
start_train = time()
pos = 0
classes = [0.0, 1.0]
while duration < 10 and pos < total_rows:
    if pos+batch_size > total_rows:
        batch_size = total_rows-pos
    X_p = f_matrix_train[pos:pos+batch_size]
    y_p = y_train.values[pos:pos+batch_size]
    model[0].partial_fit(X_p, y_p, classes)
    pos = pos + batch_size
    duration = time() - start_train
    if pos == total_rows:
        pos = 0
        batch_size = 10000
print('done')

done


In [49]:
## Сохранение результатов и расчет ошибки

In [50]:
y_test_values=y_test.values
predicted_results = model[0].predict_proba(f_matrix_test)
predicted_results = np.where(predicted_results[:,0]>predicted_results[:,1], 0.0,1.0)

sum_errors =sum(y_test_values - predicted_results)
accuracy = (len(y_test_values) - sum_errors) / len(y_test_values) *100
print('count test values', len(y_test_values))
print('sum_errors', sum_errors)
print('accuracy', accuracy)

count test values 4324
sum_errors 638.0
accuracy 85.24514338575393


In [51]:
saved_result = pd.DataFrame({'text':test.values,
 'expected':  y_test_values,
 'predicted': predicted_results})

In [52]:
saved_result.to_csv('result.csv', encoding='utf-8', index=False)